# YOLOv8 차량 번호판 학습 노트북

이 노트북은 `train_license_plate.py`와 동일한 설정으로 Ultralytics YOLOv8 모델을 학습하도록 구성되어 있습니다.
필요에 따라 학습 파라미터를 조정한 뒤 셀을 실행하세요.

In [ ]:
from pathlib import Path
import os
import sys

# 노트북에서 가상환경 패키지를 사용하도록 경로를 등록합니다.
VENV_DIR = Path('venv')
if not VENV_DIR.exists():
    raise FileNotFoundError(f'가상환경을 찾을 수 없습니다: {VENV_DIR}')

site_packages = sorted((VENV_DIR / 'lib').glob('python*/site-packages'))
if not site_packages:
    raise FileNotFoundError('site-packages 디렉터리를 찾을 수 없습니다. 가상환경 구성을 확인하세요.')
site_packages_path = site_packages[0].resolve()
if str(site_packages_path) not in sys.path:
    sys.path.insert(0, str(site_packages_path))

os.environ['VIRTUAL_ENV'] = str(VENV_DIR.resolve())
bin_path = (VENV_DIR / 'bin').resolve()
current_path = os.environ.get('PATH', '')
if current_path:
    paths = current_path.split(':')
    if str(bin_path) not in paths:
        os.environ['PATH'] = str(bin_path) + ':' + current_path
else:
    os.environ['PATH'] = str(bin_path)

print('가상환경 경로 설정 완료:', VENV_DIR)
print('현재 커널 파이썬:', sys.executable)
venv_python = bin_path / 'python'
if venv_python.exists() and sys.executable != str(venv_python):
    print('⚠️ 현재 커널이 가상환경 파이썬이 아닙니다. 필요한 경우 Jupyter 커널을 전환하세요.')


In [ ]:
from ultralytics import YOLO
from pathlib import Path

MODEL_PATH = Path('yolov8n.pt')
DATA_PATH = Path('../Korea Car License Plate/data.yaml')
PROJECT_DIR = Path('runs')
RUN_NAME = 'license_plate_yolov8n'

if not MODEL_PATH.exists():
    raise FileNotFoundError(f'모델 가중치를 찾을 수 없습니다: {MODEL_PATH}')
if not DATA_PATH.exists():
    raise FileNotFoundError(f'데이터 설정 파일을 찾을 수 없습니다: {DATA_PATH}')

print('모델 로드 중…', MODEL_PATH)
model = YOLO(str(MODEL_PATH))
print('모델 로드 완료')

train_args = dict(
    data=str(DATA_PATH),
    epochs=100,
    batch=16,
    imgsz=640,
    project=str(PROJECT_DIR),
    name=RUN_NAME,
    workers=8,
    seed=0,
    deterministic=True,
    patience=100,
    pretrained=True,
    val=True,
    save=True,
    close_mosaic=10,
    amp=True,
)

print('학습 시작')
results = model.train(**train_args)
print('학습 완료')
results


In [ ]:
from pathlib import Path
import itertools
import csv

run_dir = Path('runs') / 'license_plate_yolov8n'
results_csv = run_dir / 'results.csv'
if results_csv.exists():
    with results_csv.open('r', encoding='utf-8') as f:
        rows = list(csv.reader(f))
    header, *data = rows
    tail = list(itertools.islice(data, max(len(data) - 5, 0), None))
    print('결과 요약 (마지막 5 에폭):')
    print(', '.join(header))
    for row in tail:
        print(', '.join(row))
else:
    print('results.csv 파일을 찾을 수 없습니다. 학습이 정상적으로 저장되었는지 확인하세요.')
